In [2]:
!pip install open3d sam2 ultralytics lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 27.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 50.6 MB/s eta 0:00:00a 0:00:01


In [10]:
import cv2
import numpy as np
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from ultralytics import YOLO
import os
import glob
import open3d as o3d

def segment_pointcloud(frame, points):
    # yolo setup
    model_path = 'yolo_model.pt'
    yolo_model = YOLO(model_path)

    # sam2 setup        
    checkpoint =  "sam2_hiera_small.pt"
    model_cfg = "sam2_hiera_s.yaml"
    predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))
    # YOLO detection and SAM2 segmentation (using existing code)
    results = yolo_model.track(source=frame, persist=True, conf=0.6, verbose=False)
    if results[0].boxes.id is None:
        print("No object detected")
    else:
        masks_list = []
        bboxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        confidences = results[0].boxes.conf.cpu().numpy().astype(float)

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            predictor.set_image(frame)
            for box in bboxes:
                input_box = np.array(box).reshape(1, 4)            
                masks, _, _ = predictor.predict(box=input_box, multimask_output=False)
                mask = (masks > 0).astype(np.uint8) * 255
                masks_list.append(mask[0])

        # Visualize segmented image
        visualization_frame = frame.copy()
        for mask in masks_list:
            # Apply mask overlay on the image
            colored_mask = np.zeros_like(frame)
            colored_mask[mask > 0] = [0, 255, 0]  # Green overlay
            visualization_frame = cv2.addWeighted(visualization_frame, 1, colored_mask, 0.5, 0)
        
        # cv2.imshow("Segmented Image", visualization_frame)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # Filter and visualize point cloud
        print(f"masks_list len: {len(masks_list)}")
        points_reshaped = points.reshape(480, 640, 3)
        #filtered_points = np.zeros_like(points_reshaped)
        for mask in masks_list:
            print("loop start")
            # Reshape points to match image dimensions (480, 640, 3)
            
            
            # Get points corresponding to the mask
            
            if 'filtered_points' not in locals():
                filtered_points = points_reshaped[mask > 0]
            else:
                filtered_points = np.vstack((filtered_points, points_reshaped[mask > 0]))
        return filtered_points
            
# Get all files from both directories
rgb_dir = "/home/netbot/Documents/Malak/Intel Realsense/strawberry_extracted_rgb"
pointcloud_dir = "/home/netbot/Documents/Malak/Intel Realsense/strawberry_extracted_pointcloud"
segmented_pcl_dir="/home/netbot/Documents/Malak/sam2/sam2/segmented_pointcloud"

rgb_files = sorted(glob.glob(os.path.join(rgb_dir, "*.png")))
pointcloud_files = sorted(glob.glob(os.path.join(pointcloud_dir, "*.txt")))

# Process each pair of files
for rgb_file, pc_file in zip(rgb_files, pointcloud_files):
    # Extract IDs to ensure they match
    rgb_id = os.path.basename(rgb_file).split('_')[-1].split('.')[0]
    pc_id = os.path.basename(pc_file).split('_')[-1].split('.')[0]
    
    if rgb_id == pc_id:
        # Load the files
        frame = cv2.imread(rgb_file)
        points = np.loadtxt(pc_file)
        
        # Process the files
        filtered_points = segment_pointcloud(frame, points)
        
        # Visualize results
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(filtered_points)
        o3d.visualization.draw_geometries([pcd])
        
        # Save the segmented point cloud
        output_file = os.path.join(segmented_pcl_dir, f"segmented_{pc_id}.ply")
        o3d.io.write_point_cloud(output_file, pcd)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
masks_list len: 1
loop start
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
masks_list len: 1
loop start
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
masks_list len: 3
loop start
loop start
loop start
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL wind

In [11]:
# Visualize the segmented pointcloud

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(filtered_points)
                
# Visualize filtered point cloud
o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
